# Import packages

In [1]:
from src.utils import read_gtf, collapse_isoforms_to_proteoforms
import polars as pl

# Load datasets

In [2]:
final_classification = pl.read_parquet("nextflow_results/V47/final_classification.parquet")

In [3]:
orfanage_gtf = read_gtf("nextflow_results/V47/orfanage/orfanage.gtf")

# Start here

In [4]:
print(f"There are {final_classification.shape[0]} transcripts in the final classification")

There are 182371 transcripts in the final classification


In [5]:
print(f"There are {final_classification.filter(pl.col("structural_category").is_in(["novel_not_in_catalog", "novel_in_catalog"])).shape[0]} novel transcripts in the final classification")

There are 103953 novel transcripts in the final classification


In [7]:
print(f"There are {orfanage_gtf.filter(pl.col("feature")=="transcript").unique("transcript_id").shape[0]} transcripts that contain CDS predicted by ORFanage")

There are 162913 transcripts that contain CDS predicted by ORFanage


In [8]:
isoforms_to_proteoforms = collapse_isoforms_to_proteoforms(orfanage_gtf)

In [9]:
f"There are {isoforms_to_proteoforms.unique("base_isoform").shape[0]} unique ORFs."

'There are 81745 unique ORFs.'

In [8]:
protein_classification = pl.read_csv("nextflow_results/V47/orfanage/SFARI.protein_classification.tsv", separator="\t")

In [22]:
n_novel_protein_isoform = protein_classification\
    .join(
        isoforms_to_proteoforms.rename({"isoform": "pb"}),
        on = "pb",
        how = "left"
    ).unique("base_isoform")\
    .filter(pl.col("protein_classification_base").is_in(["pNIC", "pNNC"]))\
    .shape[0]
print(f"There are {n_novel_protein_isoform} novel protein isoforms")

There are 57489 novel protein isoforms
